# RNN SMS Text Classifier

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


## Import Libraries

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

print(tf.__version__)

## Load the Data

Let's load the data and take a quick look at the dataset (train data and test data)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [3]:
train_data  = pd.read_csv(train_file_path, sep='\t', header = None)
train_data.columns = ["label", "message"]
train_data

,label,message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


In [4]:
test_data  = pd.read_csv(test_file_path, sep='\t', header = None)
test_data.columns = ["label", "message"]
test_data

,label,message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4179 entries, 0 to 4178
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    4179 non-null   object
 1   message  4179 non-null   object
dtypes: object(2)
memory usage: 65.4+ KB


In [6]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1392 entries, 0 to 1391
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   label    1392 non-null   object
 1   message  1392 non-null   object
dtypes: object(2)
memory usage: 21.9+ KB


In [7]:
train_data.describe()

,label,message
count,4179,4179
unique,2,3935
top,ham,"sorry, i'll call later"
freq,3619,22


In [8]:
train_data.label.value_counts()

ham     3619
spam     560
Name: label, dtype: int64

In [9]:
560/3619

0.15473887814313347

In [10]:
test_data.describe()

,label,message
count,1392,1392
unique,2,1349
top,ham,"sorry, i'll call later"
freq,1205,8


In [11]:
test_data.label.value_counts()

ham     1205
spam     187
Name: label, dtype: int64

In [12]:
187/1205

0.15518672199170125

After quick look at the dataset, overall both dataset are fine (there are no missing values, label distribution for train set and test set are evenly distributed).

## Prepare the Data

First, I have to preprocess the label column into numerical value.

In [13]:
y_train = train_data['label'].map({"spam": 1, "ham": 0})
y_train

0       0
1       0
2       0
3       0
4       0
       ..
4174    0
4175    0
4176    1
4177    1
4178    0
Name: label, Length: 4179, dtype: int64

In [14]:
y_test = test_data['label'].map({"spam": 1, "ham": 0})
y_test

0       0
1       0
2       0
3       0
4       0
       ..
1387    0
1388    0
1389    0
1390    1
1391    1
Name: label, Length: 1392, dtype: int64

Then, I have to preprocess the message column using ***Tokenizer*** class to encoding message to a list of word IDs.

In [15]:
X_train = []
for _ in train_data['message'].values:
  X_train.append(_)

X_test = []
for _ in test_data['message'].values:
  X_test.append(_)

tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)

Let's find out how many distinct words in the messages and the total number of messages.

In [16]:
max_id = len(tokenizer.word_index) # number of distinct words
dataset_size = tokenizer.document_count # total number of messages

In [17]:
max_id

7692

In [18]:
dataset_size

4179

Now, let's encode all the message from X_train and X_test to a list of word IDs and put it all encoded value to a new list its called encoded list.

In [19]:
encoded_train = []
for _ in X_train:
  encoded_train.append(tokenizer.texts_to_sequences([_])[0])

encoded_test = []
for _ in X_test:
  encoded_test.append(tokenizer.texts_to_sequences([_])[0])

In [20]:
encoded_train[:5]

[[3666,
  36,
  2482,
  44,
  142,
  4,
  401,
  766,
  78,
  6,
  725,
  23,
  1,
  93,
  55,
  6,
  162,
  19,
  1,
  460,
  54,
  176,
  78,
  1615,
  110,
  24,
  1,
  314,
  153,
  44,
  12,
  14],
 [3, 29, 281, 27, 340],
 [19,
  6,
  831,
  55,
  3667,
  3668,
  357,
  3669,
  55,
  1,
  9,
  3670,
  18,
  95,
  415,
  85,
  493,
  113,
  6,
  3671,
  41,
  3672,
  434,
  537],
 [767,
  144,
  40,
  276,
  2,
  46,
  56,
  46,
  56,
  88,
  29,
  3673,
  3674,
  341,
  95,
  3675,
  3676],
 [281,
  315,
  606,
  1,
  261,
  971,
  58,
  145,
  316,
  62,
  77,
  68,
  10,
  95,
  1616,
  197,
  140,
  75]]

In [21]:
encoded_test[:5]

[[1, 64, 8, 1776, 95, 1, 31, 1471, 85, 8, 531],
 [25, 115, 36, 119, 49, 811, 3],
 [1,
  381,
  558,
  365,
  28,
  51,
  90,
  1,
  111,
  30,
  199,
  6648,
  49,
  65,
  13,
  420,
  118,
  6,
  172,
  10],
 [3538,
  150,
  4,
  493,
  107,
  111,
  7,
  1,
  6337,
  3538,
  308,
  109,
  246,
  15,
  853,
  7,
  51,
  20],
 [127, 22, 3, 865]]

Notice that they are different lengths in encoded list. This is an issue, it cannot pass different length data into neural network model. Therefore, I have to make each list in encoded list the same length. To do this I will follow the procedure below:
- if greater than 1000 words then trim off the extra words
- if less than 1000 words add the necessary amount of 0's to make it equal to 1000.

In [22]:
from keras.preprocessing import sequence

MAXLEN = 1000

train = sequence.pad_sequences(encoded_train, MAXLEN)
test = sequence.pad_sequences(encoded_test, MAXLEN)

Let's check if it's work or not.

In [23]:
train

array([[   0,    0,    0, ...,   44,   12,   14],
       [   0,    0,    0, ...,  281,   27,  340],
       [   0,    0,    0, ..., 3672,  434,  537],
       ...,
       [   0,    0,    0, ..., 3592,  220,  191],
       [   0,    0,    0, ...,  750,    2, 1113],
       [   0,    0,    0, ..., 7692,   57,  180]], dtype=int32)

In [24]:
len(train[0])

1000

In [25]:
len(train[1])

1000

In [26]:
test

array([[   0,    0,    0, ...,   85,    8,  531],
       [   0,    0,    0, ...,   49,  811,    3],
       [   0,    0,    0, ...,    6,  172,   10],
       ...,
       [   0,    0,    0, ...,    1,   79,    3],
       [   0,    0,    0, ...,  963,  739, 1186],
       [   0,    0,    0, ...,   87, 2598,  358]], dtype=int32)

In [27]:
len(test[0])

1000

In [28]:
len(test[1])

1000

Great, it's worked!

## Train a Model

Now, create the model and train it. I'll use a word embedding layer as the first layer in the model and add a LSTM layer afterwards that feeds into a dense node to get the predicted sentiment. 

64 stands for the output dimension of the vectors generated by the embedding layer.

In [29]:
# VOCAB_SIZE = 7693
VOCAB_SIZE = max_id + 1

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 64),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [30]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 64)          492352    
_________________________________________________________________
lstm (LSTM)                  (None, 64)                33024     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 525,441
Trainable params: 525,441
Non-trainable params: 0
_________________________________________________________________


In [31]:
model.compile(loss = "binary_crossentropy", optimizer = "rmsprop", metrics = ['acc'])

history = model.fit(train, y_train, epochs = 7, validation_split = 0.2)

Epoch 1/7
105/105 [==============================] - 53s 491ms/step - loss: 0.2016 - acc: 0.9261 - val_loss: 0.0636 - val_acc: 0.9868
Epoch 2/7
105/105 [==============================] - 51s 489ms/step - loss: 0.0447 - acc: 0.9877 - val_loss: 0.0598 - val_acc: 0.9868
Epoch 3/7
105/105 [==============================] - 52s 493ms/step - loss: 0.0272 - acc: 0.9925 - val_loss: 0.0617 - val_acc: 0.9833
Epoch 4/7
105/105 [==============================] - 52s 492ms/step - loss: 0.0172 - acc: 0.9952 - val_loss: 0.0438 - val_acc: 0.9880
Epoch 5/7
105/105 [==============================] - 51s 489ms/step - loss: 0.0131 - acc: 0.9961 - val_loss: 0.0371 - val_acc: 0.9928
Epoch 6/7
105/105 [==============================] - 52s 491ms/step - loss: 0.0088 - acc: 0.9973 - val_loss: 0.0447 - val_acc: 0.9928
Epoch 7/7
105/105 [==============================] - 52s 493ms/step - loss: 0.0060 - acc: 0.9988 - val_loss: 0.0523 - val_acc: 0.9904


## Evaluate and Test 

At last, let's evaluate the model on the test data to see how well it performs.

In [32]:
results = model.evaluate(test, y_test)
print(results)

44/44 [==============================] - 4s 94ms/step - loss: 0.0572 - acc: 0.9871
[0.05719532445073128, 0.9870689511299133]


Now let’s use the model to make predictions. 

Before make prediction, I need to convert any message that I write into that form so the model can understand it. To do that I'll load the encodings from the dataset before and use them to encode this message.

In [33]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred_text = [pred_text]
  encoded_pred = []
  for _ in pred_text:
    encoded_pred.append(tokenizer.texts_to_sequences([_])[0])
  
  pred = sequence.pad_sequences(encoded_pred, MAXLEN)
  result = model.predict(pred)
  if result[0][0] < 0.5:
    label = "ham"
  else:
    label = "spam"
  prediction = [result[0][0], label]
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[6.040542e-05, 'ham']


In [34]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!


In [35]:
test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

for i in test_messages:
  print(predict_message(i))

[6.040542e-05, 'ham']
[0.6415102, 'spam']
[5.7118043e-05, 'ham']
[0.99988735, 'spam']
[0.9999113, 'spam']
[0.00014007092, 'ham']
[1.888928e-05, 'ham']
